# Experiment 2
### In this experiment we compare different metamodels on an image completion task framed as a 2D regression task 
We compare the amortised GI BNN with a convolutional conditional neural process and an amortised MFVI BNN as two baseline models. The models will be evaluated on datasets which will be treated as metadatasets in which each image will be subsampled and treated as a 2D regression dataset. The following datasets will be considered:
- MNIST
- CelebA, including an out-of-distribution qualitative evaluation on the Ellen Oscars Selfie
- CIFAR10

For each test case, a linear interpolator will be used as a lower benchmark


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import wbml.experiment
import wbml.plot
from tqdm import tqdm
import sys

torch.set_default_dtype(torch.float64)

sys.path.append("../")
import bnn_amort_inf
from bnn_amort_inf.models.likelihoods.normal import (
    NormalLikelihood,
    BernoulliLikelihood,
)
from bnn_amort_inf.models.bnn import gibnn, mfvi_bnn
from bnn_amort_inf.models.np import GridConvCNP, CNP
from bnn_amort_inf import utils

torch.set_default_dtype(torch.float64)

In [ ]:
wd = wbml.experiment.WorkingDirectory("./experiment_two", log=None, override=False)

### Handle Data and Generate Metadatasets

In [ ]:
num_datasets = 100  # note cannot be greater than 9000
meta_datasets = {}
ratio = 0.25  # proportion of context pixels in training images

mnist = datasets.MNIST(
    root=wd.root,
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
mnist_iter = iter(torch.utils.data.DataLoader(mnist, shuffle=True))
img_samp = next(mnist_iter)[0]

I, M_c = (
    next(mnist_iter)[0].squeeze(0),
    utils.dataset_utils.random_mask(ratio, img_samp)[1],
)

m_d = []
for _ in range(num_datasets):
    img = next(mnist_iter)[0].squeeze(0)

    # Convert to binary.
    img = img.round()

    mask = utils.dataset_utils.random_mask(ratio, img_samp)[1]
    x_c, y_c, x_t, y_t = utils.dataset_utils.img_for_reg(img, mask)
    m_d.append((img, mask, x_c, y_c, x_t, y_t))

meta_datasets["mnist"] = utils.dataset_utils.MetaDataset(m_d)

In [ ]:
len(meta_datasets["mnist"].datasets)

### Define and Train Models

In [ ]:
plot_training_metrics = True

In [ ]:
# convcnp = GridConvCNP(
#     x_dim=2,
#     y_dim=1,
#     embedded_dim=2,
#     cnn_chans=[128, 128, 128],
#     conv_kernel_size=5,
#     cnn_kernel_size=3,
#     res=True,
# )

# convcnp_tracker = utils.training_utils.train_metamodel(
#     convcnp,
#     meta_datasets["mnist"],
#     image=True,
#     gridconv=True,
#     lr=1e-4,
#     max_iters=5_000,
#     batch_size=16,
#     min_es_iters=150,
#     ref_es_iters=50,
#     smooth_es_iters=50,
#     es=True,
#     man_thresh=(
#         "ll",
#         2000.0,
#     ),  # manual threshold that only stops training loop if ll>man_thresh... very hacky but fine for now
# )

# if plot_training_metrics:
#     fig, axes = plt.subplots(
#         len(convcnp_tracker.keys()),
#         1,
#         figsize=(8, len(convcnp_tracker.keys()) * 4),
#         dpi=100,
#         sharex=True,
#     )

#     for ax, (key, vals) in zip(axes, convcnp_tracker.items()):
#         ax.plot(vals)
#         ax.set_ylabel(key)
#         ax.grid()

#     plt.show()

In [ ]:
agibnn = gibnn.AmortisedGIBNN(
    x_dim=2,
    y_dim=1,
    hidden_dims=[20, 20],
    in_hidden_dims=[50, 50],
    np_kl=False,
    likelihood=BernoulliLikelihood(),
)

agibnn_tracker = utils.training_utils.train_metamodel(
    agibnn,
    meta_datasets["mnist"],
    image=True,
    np_loss=True,
    lr=1e-3,
    max_iters=10_000,
    batch_size=1,
    min_es_iters=50,
    ref_es_iters=30,
    smooth_es_iters=10,
    es=False,
    # man_thresh=("elbo", -2.0e10),
)

if plot_training_metrics:
    fig, axes = plt.subplots(
        len(agibnn_tracker.keys()),
        1,
        figsize=(8, len(agibnn_tracker.keys()) * 4),
        dpi=100,
        sharex=True,
    )

    for ax, (key, vals) in zip(axes, agibnn_tracker.items()):
        ax.plot(vals)
        ax.set_ylabel(key)
        ax.grid()

    plt.show()

### Generate Test Datasets

In [ ]:
num_test = 4
num_models = 1
test_datasets = []
for _ in range(num_test):
    img = next(mnist_iter)[0].squeeze(0)
    img = img.round()
    mask = utils.dataset_utils.random_mask(0.25, img_samp)[1]
    x_c, y_c, x_t, y_t = utils.dataset_utils.img_for_reg(img, mask)
    test_datasets.append((img, mask, x_c, y_c, x_t, y_t))

In [ ]:
plot_std = True

for dataset in test_datasets:
    I, M_c, x_c, y_c, x_t, y_t = dataset
    ctx_img = utils.dataset_utils.vis_ctxt_img(M_c, I)
    # convcnp_preds = convcnp(I, M_c)
    # convcnp_pred_img = convcnp_preds.loc.detach().permute(1, 2, 0).numpy()
    # convcnp_pred_std = convcnp_preds.scale.detach().permute(1, 2, 0).numpy()
    preds = agibnn(x_c, y_c, x_test=x_t, num_samples=10)[-1]
    # agibnn_pred_img, agibnn_pred_std = utils.dataset_utils.samps_to_img_dist(
    #     agibnn_preds
    # )

    pred_dists = agibnn.likelihood(preds)
    pred_probs = pred_dists.probs.detach()
    pred_img = pred_probs.mean(0).reshape((28, 28, 1))
    pred_std = pred_probs.std(0).reshape((28, 28, 1))

    num_plots = 2 + num_models * (1 + int(plot_std))
    fig, axes = plt.subplots(1, num_plots)
    axes[0].imshow(I.permute(1, 2, 0).numpy(), cmap="gray")
    axes[0].axis(False)
    axes[1].imshow(ctx_img)
    axes[1].axis(False)
    axes[2].imshow(pred_img, cmap="gray")
    axes[2].axis(False)
    if plot_std:
        axes[3].imshow(pred_std, cmap="viridis")
        axes[3].axis(False)

    plt.show()